In [1]:
import numpy as np

In [89]:
# funzioncine ricorrenti: 
vol   = lambda a: a[0].dot(np.cross(a[1],a[2]))
#b
norm  = lambda v: np.sqrt(v.dot(v))
#
recip = lambda a: 2.0 * np.pi /vol(a) * np.array([np.cross(a[i-1],a[j-1]) for i,j in [(2,3),(3,1),(1,2)]])
#
nm    = lambda at_sc,at_0,d: tuple(
            (s*(np.diag(np.abs(np.round(recip(at_0).dot(at_sc)/2/np.pi,0)).astype(np.int32))+2)[d] for s in (-1,1))
)
#
t0_sc_frac = lambda at_0, at_sc, i,j,k: recip(at_sc).dot( np.array([i,j,k]).dot(at_0))/2./np.pi
#
inbox =      lambda at_0, at_sc,i,j,k: (
    (t0_sc_frac(at_0,at_sc,i,j,k) > -1.e-6   ).all() and 
    (t0_sc_frac(at_0,at_sc,i,j,k) < 1.0-1.e-6).all()
) 
#
trasl = lambda at_sc,at_0: filter(lambda t: inbox(at_0,at_sc,*t), ((i,j,k) 
                                        for i in range(*nm(at_sc,at_0,0)) 
                                        for j in range (*nm(at_sc,at_0,1))
                                        for k in range(*nm(at_sc, at_0,2))
                                       ))
#
frac_pos = lambda at, pos:  [(_[0],recip(at).dot(_[1])/2./np.pi) for _ in pos]
#
scale_pos = lambda fac, pos: [(_[0],_[1]/fac) for _ in pos]
#
posline = lambda v: " ".join([f"{x_:12.7f}" for x_ in v])
write_pos_iteration =  lambda pos,f: (f.write(f"{_[0]:4s}" + posline(_[1])+"\n") for _ in pos)
#
tetragonal_lattice = lambda alat,bsua, csua: alat*np.array([[1.,0.,0.],[0.,bsua,0.],[0.,0.,csua]])
#
rotate_pos  = lambda old, new, pos: [(_[0],_[1].dot(new)) for _ in frac_pos(old,pos)] 
#
def supercell_pos(at_sc, at_0, pos0):
    pos_frac0 = frac_pos(at_0, pos0) 
    return  [(_[0],(_[1]+np.array(t)).dot(at_0)) for _ in pos_frac0 for t in trasl(at_sc, at_0) ]
#
def cubic_ibrav_alat(at):
    from sys import stdout
    #
    if round(at[0].dot(at[1]),8) == 0:
        alat = norm(a[0])
        ibrav = 1
    elif round(at[0].dot(at[1])/norm(at[0])/norm(at[1]),8) == 0.5:
        alat = norm(a[0])*np.sqrt(2.)
        ibrav = 2
    elif round(at[0].dot(at[1])/norm(at[0])/norm(at[1]),8) == round(-1./3.,8):
        alat = norm(at[0])/0.8660254037844386
        ibrav = -3 
    return alat, ibrav 


def tetragonal_ibrav_alat(at):
    alat = norm(at[0])
    bsua = norm(at[1])/alat
    csua = norm(at[2])/alat 
    if round(bsua,8) == 1.0:
        ibrav = 6 
    else:
        ibrav = 8 
    return alat, bsua, csua, ibrav 
    
    
#
def cubic_alat_pos(at, pos,filename=None):
    from collections import deque
    from sys import stdout 
    #
    alat, ibrav = cubic_ibrav_alat(at)
    #
    scaled_pos = scale_pos(alat, pos)
    if filename is None:
        print (f"alat = {alat:10.6f}")
        print (f"ibrav = {ibrav:4d}" )
        print (f"nat = {len(pos):4d}\n") 
        deque(write_pos_iteration(scaled_pos, stdout), maxlen=0)
        return 1
    else:
        with open(filename,'w') as f:
            f.write(f"alat = {alat:10.6f}\n")
            f.write(f"ibrav = {ibrav:4d}\n")
            f.write(f"nat = {len(pos):4d}\n\n")
            deque(write_pos_iteration(scaled_pos,f),maxlen=0)
        return 1 
#    
def tetragonal_alat_pos(at, pos, filename=None):
    from collections import deque
    from sys import stdout
    #
    alat, bsua, csua, ibrav = tetragonal_ibrav_alat(at)
    #
    scaled_pos = scale_pos(alat,pos)
    if filename is None:
        print (f"alat = {alat:10.6f}")
        if ibrav == 8: print(f"bsua = {bsua:9.6f}")
        print (f"csua = {csua:9.6f}")
        print (f"ibrav = {ibrav:4d}")
        print (f"nat = {len(pos):4d}\n")
        deque(write_pos_iteration(scaled_pos, stdout))
        return 1
    else:
        with open(filename,'w') as f:
            f.write (f"alat = {alat:10.6f}\n")
            if ibrav == 8: f.write(f"bsua = {bsua:9.6f}\n")
            f.write(f"csua = {csua:9.6f}\n")
            f.write(f"ibrav = {ibrav:4d}\n")
            f.write(f"nat = {len(pos):4d}\n\n")
            deque(write_pos_iteration(scaled_pos, f))
            return 1
    
    
                
        
     

In [3]:
#cella primitiva diamante 
alat = 3.567 / 0.5291 
lattice0 = np.array([[0.5,0.5,0.0],[0.5,0.0,0.5],[0.0,0.5,0.5]]) * alat
recip0 = recip(lattice0)
pos = [('C',np.array([0.,0.,0.])), ('C', np.array([0.25,0.25,0.25])*alat)]

In [4]:
#supercella da 32 bcc 
lattice_sc = lattice_sc = np.array([[3,-1,-1],[-1,3,-1],[-1,-1,3]]).dot(lattice0)
carbon32 = lattice_sc, supercell_pos(lattice_sc, lattice0, pos)
#
cubic_alat_pos(*carbon32,'pippo')

1

In [127]:
at_sc111 = np.array([[0,-2,2],[2,-2,0],[1,1,1]]).dot(lattice0)
temp_pos  =  supercell_pos(at_sc111, lattice0, pos)
alat, bsua, csua, ibrav = tetragonal_ibrav_alat(at_sc111)
new_latt = tetragonal_lattice(alat, bsua, csua)
#
new_pos = rotate_pos(at_sc111, new_latt, temp_pos)
tetragonal_alat_pos(at_sc111, [(_[0],_[1]*alat*0.529177) for _ in new_pos], 'c111')

1

In [ ]:
np.array(([1,1,-1])).dot(lattice0)

In [93]:
frac_pos(at_sc111,temp_pos)

[('C', array([0.25, 0.75, 0.5 ])),
 ('C', array([0.25, 0.25, 0.5 ])),
 ('C', array([1.59905252e-17, 5.00000000e-01, 0.00000000e+00])),
 ('C', array([0., 0., 0.])),
 ('C', array([0.75, 0.75, 0.5 ])),
 ('C', array([0.75, 0.25, 0.5 ])),
 ('C', array([0.5, 0.5, 0. ])),
 ('C', array([5.00000000e-01, 1.59905252e-17, 0.00000000e+00])),
 ('C', array([0.25, 0.75, 0.25])),
 ('C', array([0.25, 0.25, 0.25])),
 ('C', array([-5.22852764e-17,  5.00000000e-01, -2.50000000e-01])),
 ('C', array([ 1.35873339e-17,  2.44634152e-17, -2.50000000e-01])),
 ('C', array([0.75, 0.75, 0.25])),
 ('C', array([0.75, 0.25, 0.25])),
 ('C', array([ 0.5 ,  0.5 , -0.25])),
 ('C', array([ 5.00000000e-01, -3.02250526e-17, -2.50000000e-01]))]

In [25]:
vol(at_sc111)/vol(lattice0)

8.0

In [35]:
norm(at_sc111[2])/4

2.9192143406719833

In [61]:
p = [(_[1]+np.array(t)).dot(lattice0) for _ in frac_pos(lattice0,pos) for t in trasl(at_sc111,lattice0) ]
f = [recip(at_sc111).dot(_)/2/np.pi for _ in p ]


[array([0., 0., 0.]),
 array([-1.59905252e-17,  5.00000000e-01,  0.00000000e+00]),
 array([ 5.00000000e-01, -1.59905252e-17,  0.00000000e+00]),
 array([0.5, 0.5, 0. ]),
 array([0.25      , 0.25      , 0.61237244]),
 array([0.25      , 0.75      , 0.61237244]),
 array([0.75      , 0.25      , 0.61237244]),
 array([0.75      , 0.75      , 0.61237244]),
 array([ 1.35873339e-17, -2.44634152e-17,  3.06186218e-01]),
 array([-5.22852764e-17,  5.00000000e-01,  3.06186218e-01]),
 array([5.00000000e-01, 3.02250526e-17, 3.06186218e-01]),
 array([0.5       , 0.5       , 0.30618622]),
 array([0.25      , 0.25      , 0.91855865]),
 array([0.25      , 0.75      , 0.91855865]),
 array([0.75      , 0.25      , 0.91855865]),
 array([0.75      , 0.75      , 0.91855865])]

In [52]:
r = recip(at_sc111)

In [62]:
new_latt

array([[ 9.53411411,  0.        ,  0.        ],
       [ 0.        ,  9.53411411,  0.        ],
       [ 0.        ,  0.        , 11.67685736]])

In [63]:
at_sc111

array([[ 6.74163674,  6.74163674,  0.        ],
       [ 6.74163674, -6.74163674,  0.        ],
       [ 6.74163674,  6.74163674,  6.74163674]])

In [66]:
temp_pos

[('C', array([0., 0., 0.])),
 ('C', array([ 3.37081837, -3.37081837,  0.        ])),
 ('C', array([3.37081837, 3.37081837, 0.        ])),
 ('C', array([6.74163674, 0.        , 0.        ])),
 ('C', array([6.74163674, 3.37081837, 3.37081837])),
 ('C', array([10.11245511,  0.        ,  3.37081837])),
 ('C', array([10.11245511,  6.74163674,  3.37081837])),
 ('C', array([13.48327348,  3.37081837,  3.37081837])),
 ('C', array([1.68540919, 1.68540919, 1.68540919])),
 ('C', array([ 5.05622756, -1.68540919,  1.68540919])),
 ('C', array([5.05622756, 5.05622756, 1.68540919])),
 ('C', array([8.42704593, 1.68540919, 1.68540919])),
 ('C', array([8.42704593, 5.05622756, 5.05622756])),
 ('C', array([11.7978643 ,  1.68540919,  5.05622756])),
 ('C', array([11.7978643 ,  8.42704593,  5.05622756])),
 ('C', array([15.16868267,  5.05622756,  5.05622756]))]

In [74]:
norm(at_sc111[2])/norm(at_sc111[0])

1.2247448713915892

In [126]:
np.cross(at_sc111[2],at_sc111[0])/norm(at_sc111[0])/norm(at_sc111[2])

array([-0.40824829, -0.40824829,  0.81649658])

In [84]:
vol(at_sc111)/vol(lattice0)

8.0

In [91]:
nm(at_sc111, lattice0,1)

(-3, 3)

In [120]:
at_sc111[2]/norm(at_sc111[0])

array([0.70710678, 0.70710678, 0.70710678])

In [119]:
at_sc111[0].dot(at_sc111[2])

0.0

In [128]:
at_sc111

array([[-6.74163674,  6.74163674,  0.        ],
       [ 0.        ,  6.74163674, -6.74163674],
       [ 6.74163674,  6.74163674,  6.74163674]])